<h1>Import libs + env</h1>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import when
import pyspark.sql.functions as sf
import os
from datetime import datetime

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores",8).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 09:16:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


23/10/25 09:16:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


<h1>Method 1: Đọc dữ liệu của 30 file , sau đó mới tính toán </h1>

In [4]:
def etl_1_day(path ,file_name):
    df = spark.read.json(path+file_name)
    df = df.select('_source.*')
    df = df.withColumn("Type",
           when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
    df = df.select('Contract','Type','TotalDuration')
    df = df.filter(df.Type != 'Error')
    print('Finished Processing {}'.format(file_name))
    return df

In [5]:
start_time = datetime.now()

In [27]:
path = '/Users/habaokhanh/Study_BigData_Dataset/log_content/'

def clean_files(path):
    list_file = sorted(os.listdir(path))
    if '.DS_Store' in list_file:
        os.remove(os.path.join(path, '.DS_Store'))
    return list_file

list_file=clean_files(path)
file_name=list_file[0]

In [29]:
list_file

['20220401.json',
 '20220402.json',
 '20220403.json',
 '20220404.json',
 '20220405.json',
 '20220406.json',
 '20220407.json',
 '20220408.json',
 '20220409.json',
 '20220410.json',
 '20220411.json',
 '20220412.json',
 '20220413.json',
 '20220414.json',
 '20220415.json',
 '20220416.json',
 '20220417.json',
 '20220418.json',
 '20220419.json',
 '20220420.json',
 '20220421.json',
 '20220422.json',
 '20220423.json',
 '20220424.json',
 '20220425.json',
 '20220426.json',
 '20220427.json',
 '20220428.json',
 '20220429.json',
 '20220430.json']

In [28]:
file_name

'20220401.json'

In [30]:
result1 = etl_1_day(path ,file_name)
result1

Finished Processing 20220401.json


DataFrame[Contract: string, Type: string, TotalDuration: bigint]

In [32]:
type(result1)

pyspark.sql.dataframe.DataFrame

In [33]:
result1.show(10)

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|HNH579912|Truyền Hình|          254|
|HUFD40665|Truyền Hình|         1457|
|HNH572635|Truyền Hình|         2318|
|HND141717|Truyền Hình|         1452|
|HNH743103|Truyền Hình|          251|
|HNH893773|Truyền Hình|          924|
|HND083642|Truyền Hình|         1444|
|DNFD74404|Truyền Hình|          691|
|DTFD21200|Truyền Hình|         1436|
|LDFD05747|Truyền Hình|         1434|
+---------+-----------+-------------+
only showing top 10 rows



<h2>Test function</h2>

In [35]:
for i in list_file[1:]:
    file_name2 = i 
    result2 = etl_1_day(path ,file_name2)
    result1 = result1.union(result2)
    result1 = result1.cache()

result1.show()

Finished Processing 20220402.json
Finished Processing 20220403.json
Finished Processing 20220404.json
Finished Processing 20220405.json
Finished Processing 20220406.json
Finished Processing 20220407.json
Finished Processing 20220408.json
Finished Processing 20220409.json
Finished Processing 20220410.json
Finished Processing 20220411.json
Finished Processing 20220412.json
Finished Processing 20220413.json
Finished Processing 20220414.json
Finished Processing 20220415.json
Finished Processing 20220416.json
Finished Processing 20220417.json
Finished Processing 20220418.json
Finished Processing 20220419.json
Finished Processing 20220420.json
Finished Processing 20220421.json
Finished Processing 20220422.json
Finished Processing 20220423.json
Finished Processing 20220424.json
Finished Processing 20220425.json
Finished Processing 20220426.json
Finished Processing 20220427.json
Finished Processing 20220428.json
Finished Processing 20220429.json
Finished Processing 20220430.json
+---------+---

In [36]:
result1.show()

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|HNH579912|Truyền Hình|          254|
|HUFD40665|Truyền Hình|         1457|
|HNH572635|Truyền Hình|         2318|
|HND141717|Truyền Hình|         1452|
|HNH743103|Truyền Hình|          251|
|HNH893773|Truyền Hình|          924|
|HND083642|Truyền Hình|         1444|
|DNFD74404|Truyền Hình|          691|
|DTFD21200|Truyền Hình|         1436|
|LDFD05747|Truyền Hình|         1434|
|HNH063566|Truyền Hình|          687|
|HNH866786|Truyền Hình|          248|
|NBAAA1128|Truyền Hình|          247|
|HNH960439|Truyền Hình|          683|
|HNJ035736|Truyền Hình|          246|
|NTFD93673|Truyền Hình|         2288|
|HNJ063267|Truyền Hình|         2282|
|HNH790383|Truyền Hình|          906|
|THFD12466|Truyền Hình|          242|
|HNH566080|Truyền Hình|          242|
+---------+-----------+-------------+
only showing top 20 rows



In [37]:
result1 = result1.groupby('Contract','Type').sum()
result1.show()

+---------+-----------+------------------+
| Contract|       Type|sum(TotalDuration)|
+---------+-----------+------------------+
|DNH014998|Phim Truyện|            112733|
|HND486882|Phim Truyện|             67217|
|HUFD07189|Truyền Hình|             49687|
|HDFD36288|Truyền Hình|            184878|
|CTFD04401|Truyền Hình|           1398087|
|HNH954607|Phim Truyện|             15972|
|HNH855959|Truyền Hình|            171875|
|SGH034683|Truyền Hình|            623300|
|NTFD35330|Truyền Hình|            179719|
|NTFD48198|Phim Truyện|            187513|
|HNH443856|Truyền Hình|            240305|
|NAFD05338|Truyền Hình|           2181980|
|LCFD20510|Phim Truyện|             54075|
|QNFD29007|Truyền Hình|           1948926|
|SGH569599|Truyền Hình|            458407|
|SGH701752|Truyền Hình|            638470|
|HNH712164|Phim Truyện|             29025|
|PYFD01920|Truyền Hình|           1417476|
|HTFD13716|Truyền Hình|            134921|
|SGJ039473|Truyền Hình|            311834|
+---------+

In [38]:
result1 = result1.withColumnRenamed('sum(TotalDuration)','TotalDuration')
result1.show()

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|DNH014998|Phim Truyện|       112733|
|HND486882|Phim Truyện|        67217|
|HUFD07189|Truyền Hình|        49687|
|HDFD36288|Truyền Hình|       184878|
|CTFD04401|Truyền Hình|      1398087|
|HNH954607|Phim Truyện|        15972|
|HNH855959|Truyền Hình|       171875|
|SGH034683|Truyền Hình|       623300|
|NTFD35330|Truyền Hình|       179719|
|NTFD48198|Phim Truyện|       187513|
|HNH443856|Truyền Hình|       240305|
|NAFD05338|Truyền Hình|      2181980|
|LCFD20510|Phim Truyện|        54075|
|QNFD29007|Truyền Hình|      1948926|
|SGH569599|Truyền Hình|       458407|
|SGH701752|Truyền Hình|       638470|
|HNH712164|Phim Truyện|        29025|
|PYFD01920|Truyền Hình|      1417476|
|HTFD13716|Truyền Hình|       134921|
|SGJ039473|Truyền Hình|       311834|
+---------+-----------+-------------+
only showing top 20 rows



In [39]:
final = result1.groupBy("Contract").pivot("Type").sum("TotalDuration")
final.show()

+---------+--------+-----------+---------+--------+-----------+
| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+---------+--------+-----------+---------+--------+-----------+
|HTFD11598|    null|      15551|     null|    null|      42919|
|HPFD48556|      69|       null|     null|    null|    1468328|
|NBFD10014|    null|       null|     null|    null|    1596494|
|HNH619088|    null|      77275|    11361|    null|     917930|
|HNH036174|    null|      62674|     null|    null|     354879|
|DNH067877|    null|       null|     null|    null|     181308|
|SGH806190|    null|       null|     null|    null|     217779|
|HNH582022|    null|       null|     null|    null|    2209949|
|HNH795510|    null|      30197|      265|    null|    1196936|
|DNFD91557|    null|       null|     null|    null|      95567|
|SGH266908|    null|       null|     1636|    null|     112771|
|HNJ102653|    null|       null|     null|    null|     166767|
|HDFD42710|    null|       null|     nul

In [41]:
end_time = datetime.now()
time_processing = (end_time - start_time).total_seconds()

In [43]:
print('It took {} to process the data'.format(time_processing))

It took 1851.011467 to process the data


<h1>Method 2: Đọc và xử lý từng file , sau đó gộp tất cả kết quả lại và group by sum </h1>

In [44]:
def etl_1_day(path ,file_name):
    df = spark.read.json(path+file_name)
    df = df.select('_source.*')
    df = df.withColumn("Type",
            when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
    df = df.select('Contract','Type','TotalDuration')
    df = df.filter(df.Type != 'Error')
    df = df.groupBy('Contract','Type').sum()
    df = df.withColumnRenamed('sum(TotalDuration)','TotalDuration')
    result = df.groupBy("Contract").pivot("Type").sum("TotalDuration")
    print('Finished Processing {}'.format(file_name))
    return result
    

In [45]:
df_test = etl_1_day(path, file_name)

Finished Processing 20220401.json


In [46]:
df_test.show()

+---------+--------+-----------+---------+--------+-----------+
| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+---------+--------+-----------+---------+--------+-----------+
|HTFD11598|    null|       2884|     null|    null|        707|
|HPFD48556|      69|       null|     null|    null|      92976|
|NBFD10014|    null|       null|     null|    null|      84628|
|HNH619088|    null|       8456|      234|    null|      65210|
|HNH036174|    null|       null|     null|    null|       6049|
|DNH067877|    null|       null|     null|    null|       5760|
|SGH806190|    null|       null|     null|    null|       1131|
|HNH582022|    null|       null|     null|    null|      86400|
|HNH795510|    null|       5840|     null|    null|      68589|
|DNFD91557|    null|       null|     null|    null|       1640|
|SGH266908|    null|       null|     null|    null|        731|
|HNJ102653|    null|       null|     null|    null|       4586|
|HDFD42710|    null|       null|     nul

<h2>Test function</h2>

In [49]:
result1 = etl_1_day(path ,file_name)
for i in list_file[1:]:
    file_name2 = i 
    result2 = etl_1_day(path ,file_name2)
    result1 = result1.union(result2)

result1.show()

Finished Processing 20220401.json
Finished Processing 20220402.json
Finished Processing 20220403.json
Finished Processing 20220404.json
Finished Processing 20220405.json
Finished Processing 20220406.json
Finished Processing 20220407.json
Finished Processing 20220408.json
Finished Processing 20220409.json
Finished Processing 20220410.json
Finished Processing 20220411.json
Finished Processing 20220412.json
Finished Processing 20220413.json
Finished Processing 20220414.json
Finished Processing 20220415.json
Finished Processing 20220416.json
Finished Processing 20220417.json
Finished Processing 20220418.json
Finished Processing 20220419.json
Finished Processing 20220420.json
Finished Processing 20220421.json
Finished Processing 20220422.json
Finished Processing 20220423.json
Finished Processing 20220424.json
Finished Processing 20220425.json
Finished Processing 20220426.json
Finished Processing 20220427.json
Finished Processing 20220428.json
Finished Processing 20220429.json
Finished Proce

23/10/25 09:57:57 WARN DAGScheduler: Broadcasting large task binary with size 1261.4 KiB


+---------+--------+-----------+---------+--------+-----------+
| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+---------+--------+-----------+---------+--------+-----------+
|HTFD11598|    null|       2884|     null|    null|        707|
|HPFD48556|      69|       null|     null|    null|      92976|
|NBFD10014|    null|       null|     null|    null|      84628|
|HNH619088|    null|       8456|      234|    null|      65210|
|HNH036174|    null|       null|     null|    null|       6049|
|DNH067877|    null|       null|     null|    null|       5760|
|SGH806190|    null|       null|     null|    null|       1131|
|HNH582022|    null|       null|     null|    null|      86400|
|HNH795510|    null|       5840|     null|    null|      68589|
|DNFD91557|    null|       null|     null|    null|       1640|
|SGH266908|    null|       null|     null|    null|        731|
|HNJ102653|    null|       null|     null|    null|       4586|
|HDFD42710|    null|       null|     nul

In [50]:
result1 = result1.groupby('Contract').sum()
result1 = result1.withColumnRenamed('sum(Giải Trí)','RelaxDuration').\
    withColumnRenamed('sum(Phim Truyện)','MovieDuration').\
        withColumnRenamed('sum(Thiếu Nhi)','ChildDuration').\
            withColumnRenamed('sum(Thể Thao)','SportDuration').\
                withColumnRenamed('sum(Truyền Hình)','TVDuration')

result1.show()

23/10/25 10:00:35 WARN DAGScheduler: Broadcasting large task binary with size 1309.5 KiB
23/10/25 10:00:44 WARN DAGScheduler: Broadcasting large task binary with size 1215.0 KiB


+---------+-------------+-------------+-------------+-------------+----------+
| Contract|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|
+---------+-------------+-------------+-------------+-------------+----------+
|HTFD11598|         null|        15551|         null|         null|     42919|
|HPFD48556|           69|         null|         null|         null|   1468328|
|NBFD10014|         null|         null|         null|         null|   1596494|
|HNH619088|         null|        77275|        11361|         null|    917930|
|HNH036174|         null|        62674|         null|         null|    354879|
|DNH067877|         null|         null|         null|         null|    181308|
|SGH806190|         null|         null|         null|         null|    217779|
|HNH582022|         null|         null|         null|         null|   2209949|
|HNH795510|         null|        30197|          265|         null|   1196936|
|DNFD91557|         null|         null|         null